# Important notes

 How to run the code?

  - 1) To run the code first you need to upload the WellXplain.csv into the path this code is. Also you need to assign use_auth_token variable with the huggingface token to access the models. To get this access it may take few hours when you request it in your huggingface account. So you need to request it in advance before runing the code.
  - 2) Set classifier_index variable which represents what method (model) are you going to consider.
  - 3) you can set the ran_index with three values which provide three different random_state for data sampling.
  - 4) run the code now in the order the cells appear.
 note that the possible values for each variable is provided in the line the the variable is assigned.
 - 5) We used A100 GPU on Google Colab for runing this code. We tried **Colab Pro**.

In [65]:
from transformers import AutoModel, AutoTokenizer
from transformers import XLNetModel, XLNetTokenizer

In [66]:
import pandas as pd
import numpy as np

In [67]:
data_new=pd.read_csv('WellXplain.csv')
data=pd.DataFrame()
data['Text']=data_new['Text']
data['Aspect']=data_new['Aspect']
data['Aspect1']=data_new['Aspect']
data['Aspect2']=data_new['Aspect']
data['Aspect3']=data_new['Aspect']
data['Aspect4']=data_new['Aspect']
data['Explanations']=data_new['Explanations']

for i in range(1,5):
  if i!=1:
    data['Aspect1']=data['Aspect1'].replace(i,0)

for i in range(1,5):
  if i==2:
    data['Aspect2']=data['Aspect2'].replace(i,1)
  else:
    data['Aspect2']=data['Aspect2'].replace(i,0)

for i in range(1,5):
  if i==3:
    data['Aspect3']=data['Aspect3'].replace(i,1)
  else:
    data['Aspect3']=data['Aspect3'].replace(i,0)

for i in range(1,5):
  if i==4:
    data['Aspect4']=data['Aspect4'].replace(i,1)
  else:
    data['Aspect4']=data['Aspect4'].replace(i,0)


In [68]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
EPOCHS = 5
LEARNING_RATE= 1e-05

dimension = 4
dimensions_list = [ 'Aspect1', 'Aspect2', 'Aspect3', 'Aspect4']

# threshold = 1 #
Classifiers = ["nghuyong/ernie-2.0-en", "bert-base-uncased","roberta-base" ,"emilyalsentzer/Bio_ClinicalBERT", "xlnet-base-cased",'nlptown/bert-base-multilingual-uncased-sentiment', "mental/mental-bert-base-uncased"]
Classifiers_Abs = ["ERNIE", "BERT", "RoBERTa", "ClinicalBERT", "XLNET", "PsychBERT", "Mental-BERT"]

classifier_index = 0 #set as needed [0:"ERNIE", 1:"BERT", 2:"RoBERTa", 3:"ClinicalBERT", 4:"XLNET", 5:"PsychBERT", 6:"Mental-BERT"]
use_auth_token = '' # You have to assign your huggingface access token here

TheClassifier = Classifiers[classifier_index]
TheClassifier_Abstract = Classifiers_Abs[classifier_index]

rand_states = [345, 200, 546]
ran_index = 2
rand_state = rand_states[ran_index]

target_List = dimensions_list

import torch
device=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
class CustomDataset(torch.utils.data.Dataset):
  def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = df['Text']
        self.targets = self.df[target_List].values
        self.max_len = max_len

  def __len__(self):
        return len(self.title)

  def __getitem__(self, index):
        # print(index,":",self.title[index])
        # print(self.title[index])
        title = self.title[index]
        title = " ".join(title.split())
        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index])
        }


In [69]:
if classifier_index in [0,1,2,3]:
  tokenizer=AutoTokenizer.from_pretrained(TheClassifier)
  class BERTClass(torch.nn.Module):
      def __init__(self):
          super(BERTClass, self).__init__()
          self.bert_model = model = AutoModel.from_pretrained(TheClassifier, output_hidden_states=True, output_attentions=True, return_dict=True)# BertModel.from_pretrained('bert-base-uncased', return_dict=True)
          self.dropout = torch.nn.Dropout(0.3)
          self.linear = torch.nn.Linear(768, dimension)

      def forward(self, input_ids, attn_mask, token_type_ids):
          output = self.bert_model(
              input_ids,
              attention_mask=attn_mask,
              token_type_ids=token_type_ids
          )
          output_with_attention = output
          output_dropout = self.dropout(output.pooler_output)
          output = self.linear(output_dropout)
          return output, output_with_attention

  model = BERTClass()
  model.to(device)

if classifier_index==4:
  tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")
  class XLNETClass(torch.nn.Module):
    def __init__(self):
        super(XLNETClass, self).__init__()
        self.xlnet_model = XLNetModel.from_pretrained("xlnet-base-cased", output_hidden_states=True, output_attentions=True, return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, dimension)

    def forward(self, input_ids, attn_mask, token_type_ids):
        output_dict = self.xlnet_model(
            input_ids=input_ids,
            attention_mask=attn_mask,
            token_type_ids=token_type_ids,
            output_hidden_states=True,
            return_dict=True
        )
        last_hidden_state = output_dict.last_hidden_state
        attention_weights = output_dict.attentions
        output_dropout = self.dropout(last_hidden_state[:, -1, :])
        output = self.linear(output_dropout)
        return output, attention_weights[-1]

  model = XLNETClass()
  model.to(device)
elif classifier_index==5:
  tokenizer=AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
  class PsychBERTClass(torch.nn.Module):
    def __init__(self):
        super(PsychBERTClass, self).__init__()
        self.tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
        self.model = AutoModel.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment',output_hidden_states=True, output_attentions=True, return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768,dimension)

    def forward(self, input_ids, attn_mask, seg_ids):
        output = self.model(
            input_ids=input_ids,
            attention_mask=attn_mask,
            token_type_ids=seg_ids
        )
        output_with_attention = output
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output, output_with_attention

  model = PsychBERTClass()
  model.to(device)

elif classifier_index==6:

  # !huggingface-cli login
  from huggingface_hub import login

  login(token=use_auth_token)
  tokenizer=AutoTokenizer.from_pretrained('mental/mental-bert-base-uncased', use_auth_token=use_auth_token) # use_auth_token has to be assign to huggingface key to access the mental bert model
  class MentalBERTClass(torch.nn.Module):
    def __init__(self):
        super(MentalBERTClass, self).__init__()
        self.tokenizer = AutoTokenizer.from_pretrained('mental/mental-bert-base-uncased')
        self.model = AutoModel.from_pretrained('mental/mental-bert-base-uncased',output_hidden_states=True, output_attentions=True, return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, dimension)

    def forward(self, input_ids, attn_mask, seg_ids):
        output = self.model(
            input_ids=input_ids,
            attention_mask=attn_mask,
            token_type_ids=seg_ids
        )
        output_with_attention = output
        output_dropout = self.dropout(output.last_hidden_state[:, 0])
        output = self.linear(output_dropout)
        return output, output_with_attention

  model = MentalBERTClass()
  model.to(device)

In [70]:
test_size = 0.2
val_df = data.sample(frac=test_size, random_state=rand_state).reset_index (drop=True)
train_df = data.drop (val_df.index).reset_index (drop=True)



train_dataset=CustomDataset(train_df, tokenizer, MAX_LEN)
valid_dataset=CustomDataset(val_df, tokenizer,MAX_LEN)

train_data_loader = torch.utils.data.DataLoader (
train_dataset,
shuffle=True,
batch_size=TRAIN_BATCH_SIZE,
num_workers=0
)
val_data_loader = torch.utils.data.DataLoader (
valid_dataset,
shuffle=True,
batch_size=VALID_BATCH_SIZE,
num_workers=0
)

def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss
    return model, optimizer, checkpoint['epoch'], valid_loss_min.item()

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    print("checkpoint_path:",checkpoint_path)
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)


In [71]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)



val_targets=[]
val_outputs=[]

def train_model(n_epochs, training_loader, validation_loader, model,
                optimizer, checkpoint_path, best_model_path):

  # initialize tracker for minimum validation loss
  valid_loss_min = np.Inf

  for epoch in range(1, n_epochs+1):
    train_loss = 0
    valid_loss = 0

    model.train()

    for batch_idx, data in enumerate(training_loader):
        # print(data['input_ids'])
        ids = data['input_ids'].to(device, dtype = torch.long)
        # print(ids)
        # raise KeyboardInterrupt
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs, _ = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)

        # print(outputs)
        # loss2 = loss_fn2(outputs, targets)

        # print("loss gambler: ",loss)
        # print("reservation: ", reservationn)

        # print("loss2 CE: ", loss2)

        # raise KeyboardInterrupt
        # tar, outp = loss_fn(outputs, targets.type(torch.int64))
        # return tar, outp

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # print('before loss data in training', loss.item(), train_loss)
        train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
        #print('after loss data in training', loss.item(), train_loss)

    # print('############# Epoch {}: Training End     #############'.format(epoch))

    # print('############# Epoch {}: Validation Start   #############'.format(epoch))
    ######################
    # validate the model #
    ######################

    model.eval()

    with torch.no_grad():
      for batch_idx, data in enumerate(validation_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs, _ = model(ids, mask, token_type_ids)

            loss= loss_fn(outputs, targets)#.type(torch.int64)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
            val_targets.extend(targets.cpu().detach().numpy().tolist())
            val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

      # print('############# Epoch {}: Validation End     #############'.format(epoch))
      # calculate average losses
      #print('before cal avg train loss', train_loss)
      train_loss = train_loss/len(training_loader)
      valid_loss = valid_loss/len(validation_loader)
      # print training/validation statistics
      print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
            epoch,
            train_loss,
            valid_loss
            ))

      # create checkpoint variable and add important data
      checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
      }
        # save checkpoint
      # save_ckp(checkpoint, False, checkpoint_path, best_model_path)

      ## TODO: save the model if validation loss has decreased
      if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
        # save checkpoint as best model
        # save_ckp(checkpoint, True, checkpoint_path, best_model_path)
        valid_loss_min = valid_loss

    # print('############# Epoch {}  Done   #############\n'.format(epoch))

  return model

In [72]:
import shutil, sys

model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, "ckpt_path/themodel3.pt", "thebestone3.pt")

Epoch: 1 	Avgerage Training Loss: 0.002834 	Average Validation Loss: 0.010760
Validation loss decreased (inf --> 0.010760).  Saving model ...
Epoch: 2 	Avgerage Training Loss: 0.001880 	Average Validation Loss: 0.008310
Validation loss decreased (0.010760 --> 0.008310).  Saving model ...
Epoch: 3 	Avgerage Training Loss: 0.001424 	Average Validation Loss: 0.007566
Validation loss decreased (0.008310 --> 0.007566).  Saving model ...
Epoch: 4 	Avgerage Training Loss: 0.001051 	Average Validation Loss: 0.006887
Validation loss decreased (0.007566 --> 0.006887).  Saving model ...
Epoch: 5 	Avgerage Training Loss: 0.000842 	Average Validation Loss: 0.007401


In [73]:
final_list=[]
last_layer_attentions = []
token_scores =[]
for i in val_df['Text']:
  example = i
  encodings = tokenizer.encode_plus(
      example,
      None,
      add_special_tokens=True,
      max_length=MAX_LEN,
      padding='max_length',
      return_token_type_ids=True,
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt'
  )
  model.eval()
  with torch.no_grad():
      input_ids = encodings['input_ids'].to(device, dtype=torch.long)
      attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
      token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
      output,output_with_attention = model(input_ids, attention_mask, token_type_ids)
      final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
      # print(train_df.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])
      # final=[0 if i<0.4 else 1 for i in final_output[0]]
      # print("final",final)
      input_id_list = input_ids[0].tolist()
      tokens = tokenizer.convert_ids_to_tokens(input_id_list)
      # raise KeyBoardInteruupt
      #model_view(output_with_attention.attentions,tokens)

      final_list.append(final_output[0])
      if classifier_index in [0,1,2,3,5]:
        attentions = output_with_attention.attentions[11][0][0]

      elif classifier_index==4:
        attentions = output_with_attention[0][11]

      else:
        attentions = output_with_attention.attentions[0]

      for sample in attentions:
        last_layer_attentions.append((sample[11]).detach().cpu())  # Detach and move to CPU


      # Calculate attention scores for each token
      attention_scores = torch.sum(attentions, dim=0)
      # Normalize attention scores
      normalized_scores = attention_scores / torch.sum(attention_scores)



      # Associate each score with its corresponding token
      token_score = {}
      for j in range(len(normalized_scores)):
          token = tokenizer.convert_ids_to_tokens(input_ids.squeeze(0))[j]
          token_score[token] = normalized_scores[j].item()

      token_scores.append(token_score)

In [74]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
top = 4
Correct_samples = np.zeros((len(token_scores)))
for i in range(len(token_scores)):
  mytoken = token_scores[i]
  token_remove = []
  Explanation = (val_df.iloc[i]['Explanations']).split()
  for tok in mytoken.keys():
    if  tok in ['[SEP]', '[CLS]']:
      token_remove.append(tok)
  for item in token_remove:
    del mytoken[item]
  token_sorted = dict(sorted(mytoken.items(), key=lambda x:x[1], reverse=True))

  token_sorted_top = dict(list(token_sorted.items())[0:top])
  # print(token_sorted_top)
  # raise KeyboardInterrupt

  common_token = 0
  for item in Explanation:
    if item in token_sorted_top.keys():
      common_token += 1
  # print(Explanation, token_sorted_top, common_token)
  if common_token/len(Explanation)>.5:
    Correct_samples[i] = 1
  # raise KeyboardInterrupt
print(TheClassifier)
print('AO score:', sum(Correct_samples)/len(token_scores))
print('Number of samples with the ground truth explanations:',sum(Correct_samples),'Number of total samples:', len(token_scores))

nghuyong/ernie-2.0-en
AO score: 0.2588996763754045
Number of samples with the ground truth explanations: 160.0 Number of total samples: 618


In [75]:
last_layer_attentions = []
for batch_idx, data in enumerate(val_data_loader):
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        outputs, output_with_attention = model(ids, mask, token_type_ids)

        if classifier_index==4:
           attentions = output_with_attention # For XLNET
        else:
           attentions = output_with_attention.attentions[0]

        for sample in attentions:
          last_layer_attentions.append((sample[11]).cpu())
        # Clear GPU memory
        del ids, mask, token_type_ids, outputs, output_with_attention
        torch.cuda.empty_cache()

from numpy.linalg import svd
from numpy.linalg import matrix_rank

d=[item.detach().numpy() for item in last_layer_attentions]
U, S, VT = svd(d)
print('***********************************')
print("Model:", TheClassifier_Abstract)
print("Epochs:", EPOCHS)
print("Batch_size:",TRAIN_BATCH_SIZE)
print("Max_len:", MAX_LEN)
print("Learning Rate:", LEARNING_RATE)
print("Rand_state:", rand_state)
print("SVD_ranking :", matrix_rank(S))

***********************************
Model: ERNIE
Epochs: 5
Batch_size: 16
Max_len: 128
Learning Rate: 1e-05
Rand_state: 546
SVD_ranking : 71


In [76]:
val_list=[]
for i in range(len(val_df)):
  val_list.append(val_df[target_List][i:i+1].values.tolist()[0])

In [77]:
def finalLabels(predicted_list,val_list):
  for i in range(len(predicted_list)):
    indices=np.array(predicted_list[i]).argsort()[::-1][:int(sum(val_list[i]))]
    # argsort()[:-1][:n]
    # print(predicted_list,np.array(predicted_list[i]).argsort()[::-1][:int(sum(val_list[i]))])
    for j in range(len(predicted_list[i])):
      if j in indices:
        predicted_list[i][j]=1.0
      else:
        predicted_list[i][j]=0.0
  return predicted_list

In [78]:
finalLabels(final_list,val_list)

[[0.0, 0.0, 1.0, 0.0],
 [1.0, 0.0, 0.0, 0.0],
 [0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0],
 [1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 0.0, 1.0],
 [0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0],
 [0.0, 1.0, 0.0, 0.0],
 [0.0, 1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 1.0],
 [0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0,

In [79]:
from sklearn.metrics import classification_report
label_names = target_List
print(classification_report(val_list, final_list,target_names=label_names))

              precision    recall  f1-score   support

     Aspect1       0.95      0.38      0.54       165
     Aspect2       0.74      0.98      0.84       106
     Aspect3       0.89      0.98      0.94       242
     Aspect4       0.65      0.90      0.76       105

   micro avg       0.81      0.81      0.81       618
   macro avg       0.81      0.81      0.77       618
weighted avg       0.84      0.81      0.78       618
 samples avg       0.81      0.81      0.81       618



In [80]:
!pip install tensorflow-addons==0.16.1
import tensorflow_addons as tfa

In [81]:
import tensorflow_addons as tfa
from sklearn.metrics import matthews_corrcoef

metric = tfa.metrics.MatthewsCorrelationCoefficient(num_classes=dimension)
metric.update_state(val_list,final_list)
result = metric.result()
print(result.numpy())
print('target_List:', target_List)

val = pd.DataFrame(val_list, columns = target_List)
fin = pd.DataFrame(final_list, columns = target_List)

from sklearn.metrics import matthews_corrcoef
print(matthews_corrcoef(val["Aspect1"],fin["Aspect1"]))

from sklearn.metrics import matthews_corrcoef
print(matthews_corrcoef(val["Aspect2"],fin["Aspect2"]))

from sklearn.metrics import matthews_corrcoef
print(matthews_corrcoef(val["Aspect3"],fin["Aspect3"]))

from sklearn.metrics import matthews_corrcoef
print(matthews_corrcoef(val["Aspect4"],fin["Aspect4"]))

0.7525
target_List: ['Aspect1', 'Aspect2', 'Aspect3', 'Aspect4']
0.5323008064209725
0.816430535741492
0.8960964706232821
0.7120144186016257
